In [ ]:
#Q.No.1> How do you install Detectron2 using pip and check the version of Detectron2?
%pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu118/torch1.13/index.html

import detectron2
print(detectron2.__version__)

In [ ]:
#Q.No.2> How do you perform inference with Detectron2 using an online image?

import cv2
import requests
import numpy as np
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# Step 1: Download the image from a URL
url = "https://wallup.net/wp-content/uploads/2016/01/211594-nature-landscape.jpg"  # Replace with your image URL
response = requests.get(url)
image = np.asarray(bytearray(response.content), dtype=np.uint8)
image = cv2.imdecode(image, cv2.IMREAD_COLOR)

# Step 2: Set up the Detectron2 configuration
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.DEVICE = "cuda"  # Use "cpu" if you don't have a GPU
predictor = DefaultPredictor(cfg)

# Step 3: Perform inference
outputs = predictor(image)

# Step 4: Visualize the results
v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
result_image = v.get_image()[:, :, ::-1]

# Step 5: Display the output image
cv2.imshow("Inference Result", result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#Q.No.3>How do you visualize evaluation metrics in Detectron2, such as training loss?

import os
import cv2
import requests
import numpy as np
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.engine import DefaultPredictor
import tensorboard

# Step 1: Install and Import Required Libraries
# Install the required libraries:
# pip install detectron2 requests opencv-python tensorboard

# Step 2: Set up configuration and model
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.DEVICE = "cuda"  # Use "cpu" if no GPU available

# Step 3: Define output directory for logging
cfg.OUTPUT_DIR = "./output"  # Directory where logs will be saved
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Step 4: Set up trainer and start training
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)

# Training the model
trainer.train()

# Step 5: Visualizing with TensorBoard
# After training starts, open a terminal and run the following command:
# tensorboard --logdir=output
# Then open your browser and navigate to http://localhost:6006 to view the training logs and metrics.

# Step 6: Perform inference with an online image
# Download an image from a URL and perform inference
url = "https://wallup.net/wp-content/uploads/2016/01/211594-nature-landscape.jpg"  # Replace with your image URL
response = requests.get(url)
image = np.asarray(bytearray(response.content), dtype=np.uint8)
image = cv2.imdecode(image, cv2.IMREAD_COLOR)

# Initialize the model with the trained weights for inference
predictor = DefaultPredictor(cfg)

# Perform inference
outputs = predictor(image)

# Visualize the results
v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
result_image = v.get_image()[:, :, ::-1]

# Display the inference result
cv2.imshow("Inference Result", result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#Q.No.4> How do you run inference with TFOD2 on an online image?
import tensorflow as tf
import numpy as np
import requests
import cv2
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
from io import BytesIO

# Step 1: Load the Pretrained Model
# Load the model from a pre-trained model checkpoint (replace with your custom model path if needed)
MODEL_NAME = 'ssd_inception_v2_coco_2017_11_17'  # Example pre-trained model
PATH_TO_CKPT = f'http://download.tensorflow.org/models/object_detection/{MODEL_NAME}.tar.gz'

# Download and extract the model (for online access)
import tarfile
import os

response = requests.get(PATH_TO_CKPT)
with open('model.tar.gz', 'wb') as f:
    f.write(response.content)

with tarfile.open('model.tar.gz', 'r:gz') as tar:
    tar.extractall(path='./')

# Load the saved model
model = tf.saved_model.load('./ssd_inception_v2_coco_2017_11_17/saved_model')

# Step 2: Load Label Map
LABEL_MAP_PATH = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(LABEL_MAP_PATH, use_display_name=True)

# Step 3: Function to Load and Prepare the Image
def load_image_into_numpy_array(url):
    response = requests.get(url)
    image_data = np.array(bytearray(response.content), dtype=np.uint8)
    image = cv2.imdecode(image_data, cv2.IMREAD_COLOR)  # Decode image into an array
    return image

# Step 4: Run Inference
def run_inference(image_path):
    image_np = load_image_into_numpy_array(image_path)

    # The input needs to be a tensor, so we convert the image to a tensor
    input_tensor = tf.convert_to_tensor(image_np)
    input_tensor = input_tensor[tf.newaxis,...]  # Add batch dimension

    # Run detection
    model_fn = model.signatures['serving_default']
    output_dict = model_fn(input_tensor)

    # All outputs are batches of detections, so we take the first one.
    output_dict = {key:value.numpy() for key,value in output_dict.items()}

    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'][0],
        output_dict['detection_classes'][0].astype(np.int32),
        output_dict['detection_scores'][0],
        category_index,
        instance_masks=output_dict.get('detection_masks', None),
        use_normalized_coordinates=True,
        line_thickness=8)

    return image_np

# Step 5: Visualize Results
# Example: URL of an online image
image_url = 'https://example.com/your_image.jpg'  # Replace with your image URL

result_image = run_inference(image_url)

# Step 6: Display the result
cv2.imshow('Detection Result', result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
#Q.No.5> How do you install TensorFlow Object Detection API in Jupyter Notebook?
# Install TensorFlow
!pip install tensorflow

# Install dependencies for TensorFlow Object Detection API
!pip install tf-slim
!pip install tensorflow-hub
!pip install tensorflow-graphics
!pip install matplotlib
!pip install opencv-python

# Install the Object Detection API
!pip install --upgrade pip
!pip install setuptools==59.5.0
!pip install tensorflow-object-detection-api
# Install dependencies for building TensorFlow Object Detection API from source
!pip install pillow lxml Cython contextlib2 jupyter
!pip install matplotlib pandas opencv-python tf-slim

# Clone the TensorFlow models repository (if not already available)
!git clone https://github.com/tensorflow/models.git

# Navigate to the 'models' directory
%cd models/research/

# Install the Object Detection API from source
!python setup.py install


# Test the installation by importing the Object Detection API
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

print("TensorFlow version:", tf.__version__)
print("Object Detection API is installed successfully.")


In [ ]:
#Q.No.6>  How can you load a pre-trained TensorFlow Object Detection model?
import tensorflow as tf
from object_detection.utils import model_util
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
import numpy as np
import cv2

# URL of the pre-trained model from TensorFlow Model Zoo
MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
MODEL_PATH = 'http://download.tensorflow.org/models/object_detection/' + MODEL_NAME + '.tar.gz'

# Download and extract the model
import tarfile
import os
import requests

response = requests.get(MODEL_PATH)
with open('model.tar.gz', 'wb') as f:
    f.write(response.content)

# Extract the downloaded tar file
with tarfile.open('model.tar.gz', 'r:gz') as tar:
    tar.extractall(path='./')

# Get the path to the saved_model directory
PATH_TO_SAVED_MODEL = './ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model'
# Load the saved model
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)
# Load label map for COCO
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)
# Load an image from file (replace with the image path or URL)
image_path = 'your_image.jpg'  # Replace with your image file path
image_np = cv2.imread(image_path)
image_np = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
# Convert image to tensor
input_tensor = tf.convert_to_tensor(image_np)
input_tensor = input_tensor[tf.newaxis,...]  # Add batch dimension

# Run inference
output_dict = detect_fn(input_tensor)

# The output dictionary contains:
# 'detection_boxes', 'detection_scores', 'detection_classes', 'num_detections'
output_dict = {key:value.numpy() for key,value in output_dict.items()}

# Visualize the results
vis_util.visualize_boxes_and_labels_on_image_array(
    image_np,
    output_dict['detection_boxes'][0],
    output_dict['detection_classes'][0].astype(np.int32),
    output_dict['detection_scores'][0],
    category_index,
    instance_masks=output_dict.get('detection_masks', None),
    use_normalized_coordinates=True,
    line_thickness=8
)

# Display the result
cv2.imshow('Detection Result', cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR))
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
#Q.No.7> How do you preprocess an image from the web for TFOD2 inference?
%pip install tensorflow opencv-python numpy requests

import tensorflow as tf
import numpy as np
import requests
import cv2

def preprocess_image_from_web(image_url, target_size=None):
    """
    Preprocess an image from a URL for TFOD2 inference.

    Args:
        image_url (str): The URL of the image to download and preprocess.
        target_size (tuple): Target size for resizing the image, e.g., (320, 320). If None, no resizing is done.

    Returns:
        tf.Tensor: Preprocessed image tensor with batch dimension.
        np.array: Original image as a NumPy array (RGB format) for visualization.
    """
    # Step 1: Download the image
    response = requests.get(image_url)
    image_data = np.array(bytearray(response.content), dtype=np.uint8)

    # Step 2: Decode the image
    image = cv2.imdecode(image_data, cv2.IMREAD_COLOR)  # Decode to BGR format
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)      # Convert to RGB format

    # Step 3: Resize the image (if target_size is provided)
    if target_size:
        image = cv2.resize(image, target_size)

    # Step 4: Normalize pixel values (if required by your model)
    # image = image / 255.0  # Uncomment if the model requires normalization

    # Step 5: Convert to TensorFlow tensor and add batch dimension
    input_tensor = tf.convert_to_tensor(image, dtype=tf.uint8)  # Use dtype=tf.float32 if normalized
    input_tensor = tf.expand_dims(input_tensor, axis=0)  # Add batch dimension

    return input_tensor, image

# Example usage:
image_url = 'https://wallup.net/wp-content/uploads/2016/01/211594-nature-landscape.jpg'  # Replace with your image URL
target_size = (320, 320)  # Replace with the size required by your model

input_tensor, original_image = preprocess_image_from_web(image_url, target_size)

print(f"Preprocessed Tensor Shape: {input_tensor.shape}")


In [ ]:
#Q.No.8> How do you visualize bounding boxes for detected objects in TFOD2 inference?
import tensorflow as tf
import numpy as np
import cv2
from object_detection.utils import visualization_utils as vis_util
from object_detection.utils import label_map_util

def visualize_detections(image, output_dict, category_index):
    """
    Visualizes bounding boxes on an image.

    Args:
        image (numpy.ndarray): The original image in RGB format.
        output_dict (dict): Model output containing detection boxes, classes, and scores.
        category_index (dict): Dictionary mapping class IDs to class names.

    Returns:
        numpy.ndarray: Image with bounding boxes drawn.
    """
    vis_util.visualize_boxes_and_labels_on_image_array(
        image,
        output_dict['detection_boxes'],
        output_dict['detection_classes'].astype(np.int32),
        output_dict['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        line_thickness=5
    )
    return image

# Example Usage
if __name__ == "__main__":
    # Example detection output (replace with actual output from model inference)
    output_dict = {
        'detection_boxes': np.array([[0.1, 0.1, 0.5, 0.5], [0.6, 0.6, 0.9, 0.9]]),  # [ymin, xmin, ymax, xmax]
        'detection_classes': np.array([1, 3]),  # Class IDs (e.g., person, car, etc.)
        'detection_scores': np.array([0.95, 0.8])  # Confidence scores
    }

    # Load an image (replace with your image path)
    image_path = 'example_image.jpg'  # Replace with the actual image path
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

    # Load category index (e.g., COCO label map)
    label_map_path = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
    category_index = label_map_util.create_category_index_from_labelmap(label_map_path, use_display_name=True)

    # Visualize detections
    image_with_detections = visualize_detections(image, output_dict, category_index)

    # Display the image with detections
    image_with_detections = cv2.cvtColor(image_with_detections, cv2.COLOR_RGB2BGR)  # Convert back to BGR for OpenCV
    cv2.imshow('Detections', image_with_detections)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [ ]:
#Q.No.9> How do you define classes for custom training in TFOD2?
import tensorflow as tf

def resize_image_tf(image, target_size):
    """
    Resize an image to the specified dimensions using TensorFlow.

    Args:
        image (tf.Tensor): Input image tensor in the format [height, width, channels].
        target_size (tuple): Target size as (height, width).

    Returns:
        tf.Tensor: Resized image tensor.
    """
    resized_image = tf.image.resize(image, target_size)
    return resized_image

# Example Usage
image = tf.random.uniform(shape=(480, 640, 3), minval=0, maxval=255, dtype=tf.float32)  # Dummy image
target_size = (320, 320)
resized_image = resize_image_tf(image, target_size)
print(f"Resized Image Shape: {resized_image.shape}")

     


In [ ]:
#Q.No.11> EE> How do you resize an image before detecting object1

import tensorflow as tf

def resize_image_tf(image, target_size):
    """
    Resize an image to the specified dimensions using TensorFlow.

    Args:
        image (tf.Tensor): Input image tensor in the format [height, width, channels].
        target_size (tuple): Target size as (height, width).

    Returns:
        tf.Tensor: Resized image tensor.
    """
    resized_image = tf.image.resize(image, target_size)
    return resized_image

# Example Usage
image = tf.random.uniform(shape=(480, 640, 3), minval=0, maxval=255, dtype=tf.float32)  # Dummy image
target_size = (320, 320)
resized_image = resize_image_tf(image, target_size)
print(f"Resized Image Shape: {resized_image.shape}")

     

In [ ]:
#Q.No.12>  How can you apply a color filter (e.g., red filter) to an image?
import cv2
import numpy as np

def apply_red_filter(image):
    """
    Applies a red filter to an image by enhancing the red channel.

    Args:
        image (numpy.ndarray): Input image in BGR format.

    Returns:
        numpy.ndarray: Image with a red filter applied.
    """
    # Split the image into its color channels (B, G, R)
    blue, green, red = cv2.split(image)
    
    # Set the blue and green channels to zero
    blue[:] = 0
    green[:] = 0
    
    # Merge the channels back
    red_filtered_image = cv2.merge((blue, green, red))
    return red_filtered_image

# Example Usage
image = cv2.imread('example_image.jpg')  # Load image
red_filtered_image = apply_red_filter(image)

# Display the result
cv2.imshow('Red Filtered Image', red_filtered_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
